In [ ]:
from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

## I.Import des données ##

In [ ]:
# Nettoyage manuel des espaces multiples dans le fichier
file_path = "c:\\Users\\DF6610\\Documents\\ProjectAppauto\\welddb\\welddb.data"
with open(file_path, "r") as file:
    cleaned_lines = [" ".join(line.split()) for line in file]

# Conversion des lignes nettoyées en DataFrame

data = pd.read_csv(StringIO("\n".join(cleaned_lines)), sep=" ", na_values="N", header=None)

In [ ]:
# La taille du dataset
print("Le nombre de lignes est: " + str(data.shape[0]))
print("Le nombre de colonnes est: " + str(data.shape[1]))

In [ ]:
columns = [
    "Carbon concentration / (weight%)",
    "Silicon concentration / (weight%)",
    "Manganese concentration / (weight%)",
    "Sulphur concentration / (weight%)",
    "Phosphorus concentration / (weight%)",
    "Nickel concentration / (weight%)",
    "Chromium concentration / (weight%)",
    "Molybdenum concentration / (weight%)",
    "Vanadium concentration / (weight%)",
    "Copper concentration / (weight%)",
    "Cobalt concentration / (weight%)",
    "Tungsten concentration / (weight%)",
    "Oxygen concentration / parts per million by weight",
    "Titanium concentration / parts per million by weight",
    "Nitrogen concentration / parts per million by weight",
    "Aluminium concentration / parts per million by weight",
    "Boron concentration / parts per million by weight",
    "Niobium concentration / parts per million by weight",
    "Tin concentration / parts per million by weight",
    "Arsenic concentration / parts per million by weight",
    "Antimony concentration / parts per million by weight",
    "Current / A",
    "Voltage / V",
    "AC or DC",
    "Electrode positive or negative",
    "Heat input / kJ/mm",
    "Interpass temperature / deg C",
    "Type of weld",
    "Post weld heat treatment temperature / deg C",
    "Post weld heat treatment time / hours",
    "Yield strength / MPa",
    "Ultimate tensile strength / MPa",
    "Elongation / %",
    "Reduction of Area / %",
    "Charpy temperature / deg C",
    "Charpy impact toughness / J",
    "Hardness / kg/mm2",
    "50 % FATT",
    "Primary ferrite in microstructure / %",
    "Ferrite with second phase / %",
    "Acicular ferrite / %",
    "Martensite / %",
    "Ferrite with carbide aggregate / %",
    "Weld ID",
]

In [ ]:
data.columns = columns
data.head()

## II.Groupement ID ##

In [ ]:
# Exemple de DataFrame avec une colonne d'identifiants
data_exple = {"ID": ["abcd123", "abc124", "abcd125", "def456", "def457", "ghi789", "ghi7abc"]}

df_exple = pd.DataFrame(data_exple)


# Fonction pour trouver le plus long préfixe commun
def find_longest_common_prefix(ids):
    if not ids:
        return ""

    prefix = ids[0]  # Commencer avec le premier identifiant
    for id in ids[1:]:
        while (
            not id.startswith(prefix) and prefix
        ):  # Réduire le préfixe jusqu'à ce qu'il corresponde
            prefix = prefix[:-1]
    return prefix


# Appliquer la fonction à chaque identifiant pour déterminer le préfixe commun
def get_common_prefix(current_id, ids):
    # Chercher le préfixe commun uniquement parmi ceux qui partagent un préfixe similaire
    return find_longest_common_prefix([id for id in ids if id.startswith(current_id[:3])])


# Appliquer la fonction pour chaque ID
df_exple["Common_Prefix"] = df_exple["ID"].apply(
    lambda x: get_common_prefix(x, df_exple["ID"].tolist())
)

df_exple

In [ ]:
data["Common_Prefix"] = data["Weld ID"].apply(
    lambda x: get_common_prefix(x, data["Weld ID"].tolist())
)
data["Common_Prefix"].value_counts()

In [ ]:
# df_pivot= pd.pivot_table(df,values=['colonne1',...],index=,aggfunc={'colonne1':"mean",...})

## Remarque sur les variables cibles : 
- Hardness / kg/mm² : Une mesure de la dureté du matériau soudé, qui est souvent liée à la qualité de la soudure.
- Yield strength / MPa ou Ultimate tensile strength / MPa : Ces deux mesures de résistance pourraient également servir de variable cible, car elles reflètent la capacité du matériau à résister à la déformation ou à la rupture.
- Elongation / etou Reduction of Area / % : Ces mesures sont souvent utilisées pour évaluer la ductilité et la ténacité d'un matériau, ce qui peut être pertinent pour la qualité des soudure
- Fatt 50%##

## Petit nettoyage ##

In [ ]:
# Fonction pour nettoyer les valeurs dans les colonnes numériques
def clean_numeric_values(value):
    if isinstance(value, str) and "<" in value:
        # Enlève le symbole '<' et convertit en float
        return float(value.replace("<", "").strip())
    try:
        # Essaye de convertir la valeur en float directement
        return float(value)
    except (ValueError, TypeError):
        # Si la conversion échoue (ex. si c'est une chaîne non numérique), retourne la valeur originale
        return value


# Appliquer la fonction uniquement aux 21 premières colonnes
data.iloc[:, :21] = data.iloc[:, :21].applymap(clean_numeric_values)

# Affichage des premières lignes du DataFrame nettoyé
data

## III.Missing values ##

### III.Missing values : vision générale ###

In [ ]:
data.isnull().sum()

In [ ]:
df = data

#### Remarque: Beaucoup de valeurs manquantes ####

### III.Missing values :  Sulphur concentration / (weight%) ###


In [ ]:
data[data["Sulphur concentration / (weight%)"].isnull()]

In [ ]:
# Tracer le graphique de distribution
plt.figure(figsize=(30, 6))
sns.histplot(data["Sulphur concentration / (weight%)"], bins=10, kde=True)
plt.title("Distribution de la concentration de soufre")
plt.xlabel("Concentration de soufre / (weight%)")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
df["Sulphur concentration / (weight%)"].fillna(
    df["Sulphur concentration / (weight%)"].mean(), inplace=True
)

### III.Missing values :  Phosphorus concentration / (weight%) ###

In [ ]:
df["Phosphorus concentration / (weight%)"].describe()

In [ ]:
df[df["Phosphorus concentration / (weight%)"].isnull()]

In [ ]:
m = df[df["Common_Prefix"] == "Gar&K-1975-"]["Phosphorus concentration / (weight%)"].mean()

In [ ]:
# Fill NaN values with the mean in the original DataFrame
df.loc[df["Common_Prefix"] == "Gar&K-1975-", "Phosphorus concentration / (weight%)"] = df.loc[
    df["Common_Prefix"] == "Gar&K-1975-", "Phosphorus concentration / (weight%)"
].fillna(m)

In [ ]:
df["Phosphorus concentration / (weight%)"].fillna(
    df["Phosphorus concentration / (weight%)"].mean(), inplace=True
)

### III.Missing values :  other concentrations / (weight%) ###

In [ ]:
df["Nickel concentration / (weight%)"].fillna(
    df["Nickel concentration / (weight%)"].mean(), inplace=True
)
df["Chromium concentration / (weight%)"].fillna(
    df["Chromium concentration / (weight%)"].mean(), inplace=True
)
df["Molybdenum concentration / (weight%)"].fillna(
    df["Molybdenum concentration / (weight%)"].mean(), inplace=True
)
df["Vanadium concentration / (weight%)"].fillna(
    df["Vanadium concentration / (weight%)"].mean(), inplace=True
)
df["Copper concentration / (weight%)"].fillna(
    df["Copper concentration / (weight%)"].mean(), inplace=True
)
df["Cobalt concentration / (weight%)"].fillna(
    df["Cobalt concentration / (weight%)"].mean(), inplace=True
)
df["Tungsten concentration / (weight%)"].fillna(
    df["Tungsten concentration / (weight%)"].mean(), inplace=True
)
df["Oxygen concentration / parts per million by weight"].fillna(
    df["Oxygen concentration / parts per million by weight"].mean(), inplace=True
)
df["Titanium concentration / parts per million by weight"].fillna(
    df["Titanium concentration / parts per million by weight"].mean(), inplace=True
)

In [ ]:
df["Nitrogen concentration / parts per million by weight"].fillna(
    df["Nitrogen concentration / parts per million by weight"].mean(), inplace=True
)
df["Aluminium concentration / parts per million by weight"].fillna(
    df["Aluminium concentration / parts per million by weight"].mean(), inplace=True
)
df["Boron concentration / parts per million by weight"].fillna(
    df["Boron concentration / parts per million by weight"].mean(), inplace=True
)
df["Niobium concentration / parts per million by weight"].fillna(
    df["Niobium concentration / parts per million by weight"].mean(), inplace=True
)
df["Tin concentration / parts per million by weight"].fillna(
    df["Tin concentration / parts per million by weight"].mean(), inplace=True
)
df["Arsenic concentration / parts per million by weight"].fillna(
    df["Arsenic concentration / parts per million by weight"].mean(), inplace=True
)
df["Antimony concentration / parts per million by weight"].fillna(
    df["Antimony concentration / parts per million by weight"].mean(), inplace=True
)

In [ ]:
# Convert to numeric, coercing errors to NaN
df["Nitrogen concentration / parts per million by weight"] = pd.to_numeric(
    df["Nitrogen concentration / parts per million by weight"], errors="coerce"
)

# Optionally, drop NaN values if you want to calculate the mean only for valid entries
mean_nitrogen_concentration = df["Nitrogen concentration / parts per million by weight"].mean()
print(mean_nitrogen_concentration)

In [ ]:
df["Nitrogen concentration / parts per million by weight"].fillna(
    df["Nitrogen concentration / parts per million by weight"].mean(), inplace=True
)
df["Aluminium concentration / parts per million by weight"].fillna(
    df["Aluminium concentration / parts per million by weight"].mean(), inplace=True
)
df["Boron concentration / parts per million by weight"].fillna(
    df["Boron concentration / parts per million by weight"].mean(), inplace=True
)
df["Niobium concentration / parts per million by weight"].fillna(
    df["Niobium concentration / parts per million by weight"].mean(), inplace=True
)
df["Tin concentration / parts per million by weight"].fillna(
    df["Tin concentration / parts per million by weight"].mean(), inplace=True
)
df["Arsenic concentration / parts per million by weight"].fillna(
    df["Arsenic concentration / parts per million by weight"].mean(), inplace=True
)
df["Antimony concentration / parts per million by weight"].fillna(
    df["Antimony concentration / parts per million by weight"].mean(), inplace=True
)

### III.Missing values :  current & Voltage ###

In [ ]:
df[df["Current / A"].isnull()]["Common_Prefix"].value_counts()

In [ ]:
evans_c = df[df["Common_Prefix"] == "Evans"]["Current / A"].mean()
evans_v = df[df["Common_Prefix"] == "Evans"]["Voltage / V"].mean()

In [ ]:
# Fill NaN values with the mean in the original DataFrame
df.loc[df["Common_Prefix"] == "Evans", "Current / A"] = df.loc[
    df["Common_Prefix"] == "Evans", "Current / A"
].fillna(evans_c)
df.loc[df["Common_Prefix"] == "Evans", "Voltage / V"] = df.loc[
    df["Common_Prefix"] == "Evans", "Voltage / V"
].fillna(evans_v)

In [ ]:
df["Current / A"] = df["Current / A"].fillna(df["Current / A"].mean())
df["Voltage / V"] = df["Voltage / V"].fillna(df["Voltage / V"].mean())

### III.Missing values :  AC or DC ###

In [ ]:
df[df["AC or DC"].isnull()]["Common_Prefix"].value_counts()

In [ ]:
df[df["Common_Prefix"] == "Evans"]["AC or DC"].value_counts()

In [ ]:
df.loc[df["Common_Prefix"] == "Evans", "AC or DC"] = df.loc[
    df["Common_Prefix"] == "Evans", "AC or DC"
].fillna("DC")

In [ ]:
df["AC or DC"] = df["AC or DC"].fillna("o")

### III.Missing values :  Electrode positive ou négative ###

In [ ]:
df[df["Electrode positive or negative"].isnull()]["Common_Prefix"].value_counts()

In [ ]:
df[df["Common_Prefix"] == "Evans"]["Electrode positive or negative"].value_counts()

In [ ]:
df["Electrode positive or negative"] = df["Electrode positive or negative"].fillna("+")

### III.Missing values :  Post weld heat treatment temperature / deg C et 

Post weld heat treatment time / hours  ###

In [ ]:
df[df["Post weld heat treatment time / hours"].isnull()]["Common_Prefix"].value_counts()

In [ ]:
df[df["Common_Prefix"] == "Gar&K-1975-"]["Post weld heat treatment temperature / deg C"].mean()

In [ ]:
gark_t = df[df["Common_Prefix"] == "Gar&K-1975-"][
    "Post weld heat treatment temperature / deg C"
].mean()
gark_time = df[df["Common_Prefix"] == "Gar&K-1975-"][
    "Post weld heat treatment time / hours"
].mean()

In [ ]:
df["Post weld heat treatment temperature / deg C"] = df[
    "Post weld heat treatment temperature / deg C"
].fillna(gark_t)
df["Post weld heat treatment time / hours"] = df["Post weld heat treatment time / hours"].fillna(
    gark_time
)

### III.Missing values : Charpy   ###

In [ ]:
df[df["Charpy impact toughness / J"].isnull()]["Common_Prefix"].value_counts()

In [ ]:
evans_charpyJ = df[df["Common_Prefix"] == "Evans"]["Charpy impact toughness / J"].mean()
evanc_charpyC = df[df["Common_Prefix"] == "Evans"]["Charpy temperature / deg C"].mean()

In [ ]:
evans_charpyJ

In [ ]:
df.loc[df["Common_Prefix"] == "Evans", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Evans", "Charpy impact toughness / J"
].fillna(evans_charpyJ)
df.loc[df["Common_Prefix"] == "Evans", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Evans", "Charpy temperature / deg C"
].fillna(evanc_charpyC)

In [ ]:
pant_charpyJ = df[df["Common_Prefix"] == "PantK-1990-"]["Charpy impact toughness / J"].mean()
pant_charpyC = df[df["Common_Prefix"] == "PantK-1990-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "PantK-1990-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "PantK-1990-", "Charpy impact toughness / J"
].fillna(pant_charpyJ)
df.loc[df["Common_Prefix"] == "PantK-1990-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "PantK-1990-", "Charpy temperature / deg C"
].fillna(pant_charpyC)

In [ ]:
sven_charpyJ = df[df["Common_Prefix"] == "SvenGret-1990-"]["Charpy impact toughness / J"].mean()
sven_charpyC = df[df["Common_Prefix"] == "SvenGret-1990-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "SvenGret-1990-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "SvenGret-1990-", "Charpy impact toughness / J"
].fillna(sven_charpyJ)
df.loc[df["Common_Prefix"] == "SvenGret-1990-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "SvenGret-1990-", "Charpy temperature / deg C"
].fillna(sven_charpyC)

In [ ]:
pat_charpyJ = df[df["Common_Prefix"] == "Pat-1981-"]["Charpy impact toughness / J"].mean()
pat_charpyC = df[df["Common_Prefix"] == "Pat-1981-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Pat-1981-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Pat-1981-", "Charpy impact toughness / J"
].fillna(pat_charpyJ)
df.loc[df["Common_Prefix"] == "Pat-1981-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Pat-1981-", "Charpy temperature / deg C"
].fillna(pat_charpyC)

In [ ]:
df[df["Common_Prefix"] == "EvHtIp1979-"]["Charpy impact toughness / J"].mean()

In [ ]:
evht_charpyJ = df[df["Common_Prefix"] == "EvHtIp1979-"]["Charpy impact toughness / J"].mean()
evht_charpyC = df[df["Common_Prefix"] == "EvHtIp1979-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "EvHtIp1979-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "EvHtIp1979-", "Charpy impact toughness / J"
].fillna(evht_charpyJ)
df.loc[df["Common_Prefix"] == "EvHtIp1979-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "EvHtIp1979-", "Charpy temperature / deg C"
].fillna(evht_charpyC)

In [ ]:
df[df["Common_Prefix"] == "Icici&-1992-"]["Charpy impact toughness / J"].mean()

In [ ]:
ici_charpyJ = df[df["Common_Prefix"] == "Icici&-1992-"]["Charpy impact toughness / J"].mean()
ici_charpyC = df[df["Common_Prefix"] == "Icici&-1992-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Icici&-1992-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Icici&-1992-", "Charpy impact toughness / J"
].fillna(ici_charpyJ)
df.loc[df["Common_Prefix"] == "Icici&-1992-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Icici&-1992-", "Charpy temperature / deg C"
].fillna(ici_charpyC)

In [ ]:
df[df["Common_Prefix"] == "Blond&-1984-"]["Charpy impact toughness / J"].mean()

In [ ]:
bld_charpyJ = df[df["Common_Prefix"] == "Blond&-1984-"]["Charpy impact toughness / J"].mean()
bld_charpyC = df[df["Common_Prefix"] == "Blond&-1984-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Blond&-1984-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Blond&-1984-", "Charpy impact toughness / J"
].fillna(bld_charpyJ)
df.loc[df["Common_Prefix"] == "Blond&-1984-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Blond&-1984-", "Charpy temperature / deg C"
].fillna(bld_charpyC)

In [ ]:
df[df["Common_Prefix"] == "Mart-"]["Charpy impact toughness / J"].mean()

In [ ]:
mrt_charpyJ = df[df["Common_Prefix"] == "Mart-"]["Charpy impact toughness / J"].mean()
mrt_charpyC = df[df["Common_Prefix"] == "Mart-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Mart-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Mart-", "Charpy impact toughness / J"
].fillna(mrt_charpyJ)
df.loc[df["Common_Prefix"] == "Mart-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Mart-", "Charpy temperature / deg C"
].fillna(mrt_charpyC)

In [ ]:
df[df["Common_Prefix"] == "Ditt-"]["Charpy impact toughness / J"].mean()

In [ ]:
ditt_charpyJ = df[df["Common_Prefix"] == "Ditt-"]["Charpy impact toughness / J"].mean()
ditt_charpyC = df[df["Common_Prefix"] == "Ditt-"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Ditt-", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Ditt-", "Charpy impact toughness / J"
].fillna(ditt_charpyJ)
df.loc[df["Common_Prefix"] == "Ditt-", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Ditt-", "Charpy temperature / deg C"
].fillna(ditt_charpyC)

In [ ]:
df[df["Common_Prefix"] == "Chandel&-1985W"]["Charpy impact toughness / J"].mean()

In [ ]:
chdl_charpyJ = df[df["Common_Prefix"] == "Chandel&-1985W"]["Charpy impact toughness / J"].mean()
chdl_charpyC = df[df["Common_Prefix"] == "Chandel&-1985W"]["Charpy temperature / deg C"].mean()

In [ ]:
df.loc[df["Common_Prefix"] == "Chandel&-1985W", "Charpy impact toughness / J"] = df.loc[
    df["Common_Prefix"] == "Chandel&-1985W", "Charpy impact toughness / J"
].fillna(chdl_charpyJ)
df.loc[df["Common_Prefix"] == "Chandel&-1985W", "Charpy temperature / deg C"] = df.loc[
    df["Common_Prefix"] == "Chandel&-1985W", "Charpy temperature / deg C"
].fillna(chdl_charpyC)

In [ ]:
df["Charpy impact toughness / J"] = df["Charpy impact toughness / J"].fillna(
    df["Charpy impact toughness / J"].mean()
)
df["Charpy temperature / deg C"] = df["Charpy temperature / deg C"].fillna(
    df["Charpy temperature / deg C"].mean()
)

### III. Missing Values: features to delete ###

In [ ]:
# 50 % FATT                                                1621
# Primary ferrite in microstructure / %                    1554
# Ferrite with second phase / %                            1562
# Acicular ferrite / %                                     1562
# Martensite / %                                           1563
# Ferrite with carbide aggregate / %                       1563

In [ ]:
df = df.drop(
    columns=[
        "50 % FATT",
        "Primary ferrite in microstructure / %",
        "Ferrite with second phase / %",
        "Acicular ferrite / %",
        "Martensite / %",
        "Ferrite with carbide aggregate / %",
    ]
)

In [ ]:
df

### III.Missing values: Résumé ###

In [ ]:
df.isnull().sum()

# Les variables cibles : 

- **Yield strength** / MPa
- **Ultimate tensile strength** / MPa
- **Elongation** / %
- **Reduction of Area** / %
- **Hardness** / kg/mm²


## IV. Variables Catégoriques ##

### IV. Variables Catégoriques : Introduction

In [ ]:
df.columns

In [ ]:
#  'AC or DC'
# 'Electrode positive or negative'
# 'Type of weld'
# Sont des variables catégoriques

### IV. Variables Catégoriques : AC or DC

In [ ]:
df["AC or DC"].value_counts()

In [ ]:
df_encoded_1 = pd.get_dummies(df, columns=["AC or DC"])

In [ ]:
df_encoded_1.columns

In [ ]:
df_encoded_1.drop(columns=["AC or DC_o"], inplace=True)

### IV. Variables Catégoriques : 'Electrode positive or negative'

In [ ]:
df["Electrode positive or negative"].value_counts()

In [ ]:
df_encoded_2 = pd.get_dummies(df_encoded_1, columns=["Electrode positive or negative"])

In [ ]:
df_encoded_2.columns

In [ ]:
df_encoded_2.drop(columns=["Electrode positive or negative_-"], inplace=True)

### IV. Variables Catégoriques : 'Type of Weld'

In [ ]:
df["Type of weld"].value_counts()

In [ ]:
df_encoded = pd.get_dummies(df_encoded_2, columns=["Type of weld"])

In [ ]:
df_encoded.columns

In [ ]:
df_encoded.drop(columns=["Type of weld_GMAA"], inplace=True)

## V. Analyse des composantes principales: ACP

### V. ACP : drop les variables cibles

In [ ]:
df_encoded.columns

In [ ]:
X = df_encoded[
    [
        "Carbon concentration / (weight%)",
        "Silicon concentration / (weight%)",
        "Manganese concentration / (weight%)",
        "Sulphur concentration / (weight%)",
        "Phosphorus concentration / (weight%)",
        "Nickel concentration / (weight%)",
        "Chromium concentration / (weight%)",
        "Molybdenum concentration / (weight%)",
        "Vanadium concentration / (weight%)",
        "Copper concentration / (weight%)",
        "Cobalt concentration / (weight%)",
        "Tungsten concentration / (weight%)",
        "Oxygen concentration / parts per million by weight",
        "Titanium concentration / parts per million by weight",
        "Nitrogen concentration / parts per million by weight",
        "Aluminium concentration / parts per million by weight",
        "Boron concentration / parts per million by weight",
        "Niobium concentration / parts per million by weight",
        "Tin concentration / parts per million by weight",
        "Arsenic concentration / parts per million by weight",
        "Antimony concentration / parts per million by weight",
        "Current / A",
        "Voltage / V",
        "Heat input / kJ/mm",
        "Interpass temperature / deg C",
        "Post weld heat treatment temperature / deg C",
        "Post weld heat treatment time / hours",
        "Charpy temperature / deg C",
        "Charpy impact toughness / J",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]

In [ ]:
y = df_encoded[
    [
        "Yield strength / MPa",
        "Ultimate tensile strength / MPa",
        "Elongation / %",
        "Reduction of Area / %",
        "Hardness / kg/mm2",
    ]
]

In [ ]:
X.describe()

### V. ACP : Centrer et réduire

In [ ]:
def replace_range_with_avg(value):
    if isinstance(value, str) and "-" in value:
        try:
            # Séparer les deux nombres et calculer la moyenne
            numbers = list(map(float, value.split("-")))
            return sum(numbers) / len(numbers)
        except ValueError:
            return value  # Si ce n'est pas un format attendu, garder la valeur originale
    else:
        return value  # Si ce n'est pas une chaîne au format '150-200', garder la valeur originale


# Appliquer la fonction sur toutes les colonnes du DataFrame
X = X.applymap(replace_range_with_avg)

In [ ]:
def center_and_normalize(df):
    # Initialiser le StandardScaler
    scaler = StandardScaler()

    # Appliquer le scaler sur le DataFrame (centrement et normalisation)
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return df_scaled

In [ ]:
X28_scaled = center_and_normalize(
    X[
        [
            "Carbon concentration / (weight%)",
            "Silicon concentration / (weight%)",
            "Manganese concentration / (weight%)",
            "Sulphur concentration / (weight%)",
            "Phosphorus concentration / (weight%)",
            "Nickel concentration / (weight%)",
            "Chromium concentration / (weight%)",
            "Molybdenum concentration / (weight%)",
            "Vanadium concentration / (weight%)",
            "Copper concentration / (weight%)",
            "Cobalt concentration / (weight%)",
            "Tungsten concentration / (weight%)",
            "Oxygen concentration / parts per million by weight",
            "Titanium concentration / parts per million by weight",
            "Nitrogen concentration / parts per million by weight",
            "Aluminium concentration / parts per million by weight",
            "Boron concentration / parts per million by weight",
            "Niobium concentration / parts per million by weight",
            "Tin concentration / parts per million by weight",
            "Arsenic concentration / parts per million by weight",
            "Antimony concentration / parts per million by weight",
            "Current / A",
            "Voltage / V",
            "Heat input / kJ/mm",
            "Interpass temperature / deg C",
            "Post weld heat treatment temperature / deg C",
            "Post weld heat treatment time / hours",
            "Charpy temperature / deg C",
            "Charpy impact toughness / J",
        ]
    ]
)

In [ ]:
X28_scaled

### V. ACP : Choix du nombre de composants

In [ ]:
# Appliquer la PCA
pca = PCA()  # Laisser scikit-learn décider du nombre maximal de composantes
pca.fit(X28_scaled)

# Variance expliquée par chaque composante
explained_variance_ratio = pca.explained_variance_ratio_

# Calcul de la variance cumulée
explained_variance_cumulative = np.cumsum(explained_variance_ratio)

# Tracer le graphe
plt.figure(figsize=(8, 5))
plt.plot(
    range(1, len(explained_variance_cumulative) + 1),
    explained_variance_cumulative,
    marker="",
    linestyle="-",
)
plt.title("Variance expliquée cumulée en fonction du nombre de composantes principales")
plt.xlabel("Nombre de composantes principales")
plt.ylabel("Variance expliquée cumulée")
# Ajouter une ligne horizontale pour montrer le seuil de 95% de variance expliquée
plt.axhline(y=0.9, color="r", linestyle="--", label="95% de variance expliquée")

# Ajouter des titres et des labels
plt.title("Variance expliquée cumulée en fonction du nombre de composantes principales")
plt.xlabel("Nombre de composantes principales")
plt.ylabel("Variance expliquée cumulée")

# Afficher la grille et la légende
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
# si on prend 20 composantes on aura 90% de la variance

### V. ACP

In [ ]:
# Appliquer la PCA
pca = PCA(n_components=20)
pca.fit(X28_scaled)
pca_result = pca.transform(X28_scaled)
# Créer un DataFrame pour les résultats de la PCA
pca_X = pd.DataFrame(
    pca_result,
    columns=[
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
    ],
)
# Afficher les résultats
pca_X

In [ ]:
print("Variance expliquée cumulée par chaque composante :", pca.explained_variance_ratio_.sum())

In [ ]:
X_processed = pd.concat(
    [
        pca_X,
        X[
            [
                "AC or DC_AC",
                "AC or DC_DC",
                "Electrode positive or negative_+",
                "Electrode positive or negative_0",
                "Type of weld_FCA",
                "Type of weld_GTAA",
                "Type of weld_MMA",
                "Type of weld_NGGMA",
                "Type of weld_NGSAW",
                "Type of weld_SA",
                "Type of weld_SAA",
                "Type of weld_ShMA",
                "Type of weld_TSA",
            ]
        ],
    ],
    axis=1,
)

In [ ]:
data_processed = pd.concat([X_processed, y], axis=1)
data_processed

### V. ACP : interprétation

In [ ]:
# Projection des données sur les deux premières composantes principales
plt.figure(figsize=(10, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c="blue", edgecolor="k", s=50)
plt.title("Projection des données sur les deux premières composantes principales")
plt.xlabel("Première composante principale")
plt.ylabel("Deuxième composante principale")
plt.grid(True)
plt.show()

In [ ]:
# Contribution des variables aux composantes principales
# Créer un DataFrame pour visualiser les coefficients
components_df = pd.DataFrame(
    pca.components_,
    columns=X28_scaled.columns,
    index=[f"Composante {i+1}" for i in range(pca.n_components_)],
)

# Afficher les poids des variables pour les deux premières composantes
print("Poids des variables pour les deux premières composantes principales :")
components_df

In [ ]:
# Visualiser les poids des variables pour les deux premières composantes
plt.figure(figsize=(10, 6))
plt.barh(components_df.columns, components_df.iloc[0], color="b", alpha=0.7, label="Composante 1")
plt.barh(components_df.columns, components_df.iloc[1], color="g", alpha=0.7, label="Composante 2")
plt.title("Poids des variables pour les deux premières composantes principales")
plt.xlabel("Poids")
plt.ylabel("Variables")
plt.legend()
plt.grid(True)
plt.show()

## VI. Clustering

In [ ]:
scaled_features = X_processed

In [ ]:
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(scaled_features)

# Ajouter les labels de cluster à vos données
X_processed["Cluster"] = kmeans.labels_

In [ ]:
# Appliquer t-SNE pour réduire les dimensions à 2
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(scaled_features)

# Créer un DataFrame avec les résultats t-SNE et les clusters
tsne_df = pd.DataFrame(data=tsne_results, columns=["TSNE1", "TSNE2"])
tsne_df["Cluster"] = kmeans.labels_

# Visualisation
plt.figure(figsize=(10, 6))
plt.scatter(
    tsne_df["TSNE1"],
    tsne_df["TSNE2"],
    c=tsne_df["Cluster"],
    cmap="viridis",
    marker="o",
    edgecolor="k",
    s=100,
)
plt.title("Clustering K-means avec t-SNE")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.colorbar(label="Cluster")
plt.show()

In [ ]:
data_processed["clusters"] = kmeans.labels_

In [ ]:
def extract_numeric(value):
    if isinstance(value, str):  # Vérifie si la valeur est une chaîne
        return pd.to_numeric(value.split("(")[0], errors="coerce")
    else:
        return np.nan  # Retourne NaN si ce n'est pas une chaîne

In [ ]:
data_processed["Hardness / kg/mm2"] = data_processed["Hardness / kg/mm2"].apply(extract_numeric)

In [ ]:
df_pivot = pd.pivot_table(
    data_processed,
    values=[
        "Yield strength / MPa",
        "Ultimate tensile strength / MPa",
        "Elongation / %",
        "Reduction of Area / %",
        "Hardness / kg/mm2",
    ],
    index=["clusters"],
    aggfunc="mean",
)
df_pivot

In [ ]:
data_processed[
    [
        "Yield strength / MPa",
        "Ultimate tensile strength / MPa",
        "Elongation / %",
        "Reduction of Area / %",
        "Hardness / kg/mm2",
    ]
].describe()

## VII. Regressions

In [ ]:
y.isnull().sum()

### VII. Yield strength

In [ ]:
data_processed.columns

In [ ]:
yield_data = data_processed[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
        "Yield strength / MPa",
    ]
]

In [ ]:
yield_data_test = yield_data[yield_data["Yield strength / MPa"].isnull()]
yield_data_test

In [ ]:
yield_data_model = yield_data[yield_data["Yield strength / MPa"].notnull()]
yield_data_model.columns

In [ ]:
x = yield_data_model[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
y = yield_data_model["Yield strength / MPa"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = SGDRegressor()
model.fit(X_train, y_train)

In [ ]:
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)
# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")

In [ ]:
# Afficher les hyperparamètres du modèle
hyperparameters = model.get_params()
print("Hyperparamètres du modèle :")
for param, value in hyperparameters.items():
    print(f"{param}: {value}")

In [ ]:
yield_data_test.columns

In [ ]:
new_data = yield_data_test[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
pred = model.predict(new_data)

In [ ]:
# Trouver les indices des valeurs manquantes
nan_indices = data_processed[data_processed["Yield strength / MPa"].isnull()].index

# Remplacer les valeurs manquantes avec les valeurs de 'pred'
data_processed.loc[nan_indices, "Yield strength / MPa"] = pred

### VII. Ultimate tensile strength / MPa

In [ ]:
yield_data = data_processed[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
        "Ultimate tensile strength / MPa",
    ]
]

In [ ]:
yield_data_test = yield_data[yield_data["Ultimate tensile strength / MPa"].isnull()]

In [ ]:
yield_data_model = yield_data[yield_data["Ultimate tensile strength / MPa"].notnull()]

In [ ]:
x = yield_data_model[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
y = yield_data_model["Ultimate tensile strength / MPa"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model = SGDRegressor()
model.fit(X_train, y_train)

In [ ]:
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)
# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")

In [ ]:
new_data = yield_data_test[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
pred = model.predict(new_data)

In [ ]:
# Trouver les indices des valeurs manquantes
nan_indices = data_processed[data_processed["Ultimate tensile strength / MPa"].isnull()].index

# Remplacer les valeurs manquantes avec les valeurs de 'pred'
data_processed.loc[nan_indices, "Ultimate tensile strength / MPa"] = pred

### VII. Elongation / %

In [ ]:
yield_data = data_processed[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
        "Elongation / %",
    ]
]
yield_data_test = yield_data[yield_data["Elongation / %"].isnull()]
yield_data_model = yield_data[yield_data["Elongation / %"].notnull()]
x = yield_data_model[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
y = yield_data_model["Elongation / %"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = SGDRegressor()
model.fit(X_train, y_train)
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)
# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")
new_data = yield_data_test[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
pred = model.predict(new_data)
# Trouver les indices des valeurs manquantes
nan_indices = data_processed[data_processed["Elongation / %"].isnull()].index

# Remplacer les valeurs manquantes avec les valeurs de 'pred'
data_processed.loc[nan_indices, "Elongation / %"] = pred

### VII. Reduction of area / %

In [ ]:
yield_data = data_processed[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
        "Reduction of Area / %",
    ]
]
yield_data_test = yield_data[yield_data["Reduction of Area / %"].isnull()]
yield_data_model = yield_data[yield_data["Reduction of Area / %"].notnull()]
x = yield_data_model[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
y = yield_data_model["Reduction of Area / %"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = SGDRegressor()
model.fit(X_train, y_train)
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)
# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")
new_data = yield_data_test[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
pred = model.predict(new_data)
# Trouver les indices des valeurs manquantes
nan_indices = data_processed[data_processed["Reduction of Area / %"].isnull()].index

# Remplacer les valeurs manquantes avec les valeurs de 'pred'
data_processed.loc[nan_indices, "Reduction of Area / %"] = pred

### VII. Hardness / kg/mm2

In [ ]:
yield_data = data_processed[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
        "Hardness / kg/mm2",
    ]
]
yield_data_test = yield_data[yield_data["Hardness / kg/mm2"].isnull()]
yield_data_model = yield_data[yield_data["Hardness / kg/mm2"].notnull()]
x = yield_data_model[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
y = yield_data_model["Hardness / kg/mm2"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = SGDRegressor()
model.fit(X_train, y_train)
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)
# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE : {rmse:.2f}")
new_data = yield_data_test[
    [
        "PC1",
        "PC2",
        "PC3",
        "PC4",
        "PC5",
        "PC6",
        "PC7",
        "PC8",
        "PC9",
        "PC10",
        "PC11",
        "PC12",
        "PC13",
        "PC14",
        "PC15",
        "PC16",
        "PC17",
        "PC18",
        "PC19",
        "PC20",
        "AC or DC_AC",
        "AC or DC_DC",
        "Electrode positive or negative_+",
        "Electrode positive or negative_0",
        "Type of weld_FCA",
        "Type of weld_GTAA",
        "Type of weld_MMA",
        "Type of weld_NGGMA",
        "Type of weld_NGSAW",
        "Type of weld_SA",
        "Type of weld_SAA",
        "Type of weld_ShMA",
        "Type of weld_TSA",
    ]
]
pred = model.predict(new_data)
# Trouver les indices des valeurs manquantes
nan_indices = data_processed[data_processed["Hardness / kg/mm2"].isnull()].index

# Remplacer les valeurs manquantes avec les valeurs de 'pred'
data_processed.loc[nan_indices, "Hardness / kg/mm2"] = pred

In [ ]:
data_processed.isnull().any()

## VII. Clustering quality

In [ ]:
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(
    data_processed[
        [
            "Yield strength / MPa",
            "Ultimate tensile strength / MPa",
            "Elongation / %",
            "Reduction of Area / %",
            "Hardness / kg/mm2",
        ]
    ]
)

# Ajouter les labels de cluster à vos données
data_processed["cluster_quality"] = kmeans.labels_

In [ ]:
# Appliquer t-SNE pour réduire les dimensions à 2
quality_scaled = center_and_normalize(
    data_processed[
        [
            "Yield strength / MPa",
            "Ultimate tensile strength / MPa",
            "Elongation / %",
            "Reduction of Area / %",
            "Hardness / kg/mm2",
        ]
    ]
)
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(quality_scaled)

# Créer un DataFrame avec les résultats t-SNE et les clusters
tsne_df = pd.DataFrame(data=tsne_results, columns=["TSNE1", "TSNE2"])
tsne_df["Cluster"] = kmeans.labels_

# Visualisation
plt.figure(figsize=(10, 6))
plt.scatter(
    tsne_df["TSNE1"],
    tsne_df["TSNE2"],
    c=tsne_df["Cluster"],
    cmap="viridis",
    marker="o",
    edgecolor="k",
    s=100,
)
plt.title("Clustering K-means avec t-SNE")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.colorbar(label="Cluster")
plt.show()

In [ ]:
df_pivot = pd.pivot_table(
    data_processed,
    values=[
        "Yield strength / MPa",
        "Ultimate tensile strength / MPa",
        "Elongation / %",
        "Reduction of Area / %",
        "Hardness / kg/mm2",
    ],
    index=["cluster_quality"],
    aggfunc="mean",
)
df_pivot

In [ ]:
data_processed[
    [
        "Yield strength / MPa",
        "Ultimate tensile strength / MPa",
        "Elongation / %",
        "Reduction of Area / %",
        "Hardness / kg/mm2",
    ]
].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ajouter une colonne pour la distribution globale
data_processed["cluster_global"] = "Global"

# Combiner les colonnes 'cluster_quality' et 'cluster_global' pour obtenir une vue complète
data_with_global = pd.concat(
    [
        data_processed[["cluster_quality"] + variables],
        data_processed[["cluster_global"] + variables].rename(
            columns={"cluster_global": "cluster_quality"}
        ),
    ]
)

# Liste des variables d'intérêt
variables = [
    "Yield strength / MPa",
    "Ultimate tensile strength / MPa",
    "Elongation / %",
    "Reduction of Area / %",
    "Hardness / kg/mm2",
]

# Création de sous-plots pour chaque variable
plt.figure(figsize=(15, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x="cluster_quality", y=var, data=data_with_global)
    plt.title(f"Distribution of {var} by Cluster and Globally")
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()